# Data Preprocessing

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()
import numpy as np 
import pandas as pd 
import seaborn as sns
!pip install gensim spacy nltk
import re 
import gensim
import spacy

     |████████████████████████████████| 6.6 MB 241 kB/s eta 0:00:01
     |████████████████████████████████| 107 kB 230 kB/s eta 0:00:01
     |████████████████████████████████| 761 kB 41 kB/s eta 0:00:016
     |████████████████████████████████| 2.9 MB 92 kB/s eta 0:00:014
     |████████████████████████████████| 181 kB 130 kB/s eta 0:00:01
     |████████████████████████████████| 489 kB 104 kB/s eta 0:00:01
     |████████████████████████████████| 48 kB 36 kB/s eta 0:00:01
     |████████████████████████████████| 6.1 MB 94 kB/s eta 0:00:015


In [ ]:
def clean_text(text):  
    # ref: https://github.com/bakrianoo/aravec
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى",
              "\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا",
               "","","","ي","",' ', ' ',' ',' ? ',' ؟ ', ' ! ']
    
    tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(tashkeel,"", text)
    
    # longation = re.compile(r'(.)\1+')
    # subst = r"\1\1"
    # text = re.sub(longation, subst, text)
    
    text = re.sub(r"[^\w\s]", '', text)
    text = re.sub(r"[a-zA-Z]", '', text)
    text = re.sub(r"\d+", ' ', text)
    text = re.sub(r"\n+", ' ', text)
    text = re.sub(r"\t+", ' ', text)
    text = re.sub(r"\r+", ' ', text)
    text = re.sub(r"\s+", ' ', text)
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    text = text.strip()
    
    return text

# Data Discovery and Integration

In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


ArSarcasm

In [4]:
import pandas as pd
train_df =  pd.read_csv('/content/drive/My Drive/Bachelor/ArSarcasm-master/ArSarcasm_train.csv')
# train_df.head()
test_df =  pd.read_csv('/content/drive/My Drive/Bachelor/ArSarcasm-master/ArSarcasm_test.csv')
# test_df.head()

In [6]:
train_df.isnull().mean()*100

dialect               0.0
sarcasm               0.0
sentiment             0.0
original_sentiment    0.0
tweet                 0.0
source                0.0
dtype: float64

In [7]:
test_df.isnull().mean()*100

dialect               0.0
sarcasm               0.0
sentiment             0.0
original_sentiment    0.0
tweet                 0.0
source                0.0
dtype: float64

In [8]:
train_df.head()

,dialect,sarcasm,sentiment,original_sentiment,tweet,source
0,gulf,False,negative,negative,"""نصيحه ما عمرك اتنزل لعبة سوبر ماريو مش زي ما ...",semeval
1,msa,False,neutral,positive,"""#نادين_نسيب_نجيم ❤️❤️❤️مجلة #ماري_كلير 💭#ملكة...",semeval
2,egypt,False,neutral,neutral,"""@Alito_NBA اتوقع انه بيستمر""",semeval
3,levant,True,neutral,negative,"""@KSA24 يعني ""بموافقتنا"" لأن دمشق صايرة موسكو""",semeval
4,msa,False,neutral,negative,"""RT @alaahmad20: قائد في الحرس يعترف بفقدان ال...",semeval


In [9]:
test_df.head()

,dialect,sarcasm,sentiment,original_sentiment,tweet,source
0,msa,True,negative,negative,"""@AbuEmad74241481 @Cesars2014 هههههه حزب الله ...",semeval
1,gulf,False,positive,neutral,"""RT @JannetForster: البنات اللي م صامو بقولكم ...",semeval
2,msa,True,neutral,neutral,اشارة رابعة اشبه بنار تحرق الانقلابيين,astd
3,msa,False,neutral,neutral,"""@EGYPTAIR ماهي مميزات درجه بزنس علماً اني في ...",semeval
4,msa,False,neutral,neutral,ما لا تراه على التلفاز منافسة شديدة بين المرشح...,semeval


In [10]:
list(train_df['tweet'][0:5])

['"نصيحه ما عمرك اتنزل لعبة سوبر ماريو مش زي ما كنّا متوقعين الله يرحم ايامات السيقا والفاميلي #SuperMarioRun"',
 '"#نادين_نسيب_نجيم ❤️❤️❤️مجلة #ماري_كلير 💭#ملكة_الصحراء  👑https://t.co/fVtn489TUS@nadinenjeim @MarieClaire_AR"',
 '"@Alito_NBA اتوقع انه بيستمر"',
 '"@KSA24 يعني "بموافقتنا" لأن دمشق صايرة موسكو"',
 '"RT @alaahmad20: قائد في الحرس يعترف بفقدان السيطرة الأمنية في شرقي وغربي إيران - أحوازنا#الأحواز #السعودية @abo_asseel @spagov  https:/…"']

In [11]:
train_df['cleaned_tweet'] = train_df.tweet.apply(clean_text)
test_df['cleaned_tweet'] = test_df.tweet.apply(clean_text)

#dropping columns original_sentiment and source and tweet 
train=train_df.drop(['original_sentiment', 'source','tweet'], axis=1)
test=test_df.drop(['original_sentiment', 'source','tweet'], axis=1)

#adding word count column 
train["Number of Words in tweet"] = train["cleaned_tweet"].apply(lambda n: len(n.split()))
test["Number of Words in tweet"] = test["cleaned_tweet"].apply(lambda n: len(n.split()))

arSarcasm = train.append(test, ignore_index=True)
arSarcasm.to_csv('arSarcasm.csv')

In [12]:
list(train_df['cleaned_tweet'][0:5])

['نصيحه ما عمرك اتنزل لعبه سوبر ماريو مش زي ما كنا متوقعين الله يرحم ايامات السيقا والفاميلي',
 'نادين نسيب نجيم مجله ماري كلير ملكه الصحراء',
 'اتوقع انه بيستمر',
 'يعني بموافقتنا لان دمشق صايره موسكو',
 'قائد في الحرس يعترف بفقدان السيطره الامنيه في شرقي وغربي ايران احوازنالاحواز السعوديه']

ArSarcasm-v2

In [13]:
train_df2 =  pd.read_csv('/content/drive/My Drive/Bachelor/ArSarcasm-v2-main/training_data.csv')
# train_df2.head()
test_df2 =  pd.read_csv('/content/drive/My Drive/Bachelor/ArSarcasm-v2-main/testing_data.csv')
# test_df2.head()


In [14]:
def changingSentiment(sentiment):
    if(sentiment=='NEG'):
        return "negative"
    elif(sentiment=='POS'):
        return "positive"
    else:
        return "neutral"

In [15]:
train_df2['sentiment']=train_df2.apply(lambda x: changingSentiment(x['sentiment']),axis=1)
test_df2['sentiment']=test_df2.apply(lambda x: changingSentiment(x['sentiment']),axis=1)

In [19]:
train_df2.isnull().mean()*100

tweet            0.0
sarcasm          0.0
sentiment        0.0
dialect          0.0
cleaned_tweet    0.0
dtype: float64

In [20]:
test_df2.isnull().mean()*100

tweet            0.0
sarcasm          0.0
sentiment        0.0
dialect          0.0
cleaned_tweet    0.0
dtype: float64

In [16]:
list(train_df2['tweet'][0:5])

['"د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق رقم مهم في المعادلة السياسية المصرية ولا يمكن إغفالههل ترى أن هذا صحيح؟#أربعة_زائد_واحد"',
 '"مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHbkBNu"',
 '“الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين لإلغاء التسعيرة كلاهما يريد تصفية السوق السوداء بجعلها حرة.”  #الاختلاط  ',
 '"@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasafa @reoshalm @Mnory202 مساكين من الصبح و هوما رايحين راجعين عاى غوغل تعبت بدالهم ههههه ي"',
 '"قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حلب ولا تقل حلب الغربية ....فحلب موحدة ويريدونها منقسمة #حلب_تنتصر"']

In [17]:
train_df2['cleaned_tweet'] = train_df2.tweet.apply(clean_text)
test_df2['cleaned_tweet'] = test_df2.tweet.apply(clean_text)

#dropping tweet column
train2=train_df2.drop(['tweet'], axis=1)
test2=test_df2.drop(['tweet'], axis=1)

#adding word count column 
train2["Number of Words in tweet"] = train2["cleaned_tweet"].apply(lambda n: len(n.split()))
test2["Number of Words in tweet"] = test2["cleaned_tweet"].apply(lambda n: len(n.split()))

arSarcasm2= train2.append(test2, ignore_index=True)
arSarcasm2.to_csv('arSarcasm2.csv')

In [18]:
list(train_df2['cleaned_tweet'][0:5])

['د محمود العلايلياري ان الفريق احمد شفيق رقم مهم في المعادله السياسيه المصريه ولا يمكن اغفالههل تري ان هذا صحيحاربعه زائد واحد',
 'مع فيدرر يااجا والكبار',
 'الداعون لمبدا الاختلاط بين الجنسين كالداعين لالغاء التسعيره كلاهما يريد تصفيه السوق السوداء بجعلها حره الاختلاط',
 'مساكين من الصبح وهوما رايحين راجعين عاي غوغل تعبت بدالهم ههههه ي',
 'قل شرق حلب ولا تقل حلب الشرقيه وقل غرب حلب ولا تقل حلب الغربيه فحلب موحده ويريدونها منقسمه حلب تنتصر']

Sarcasm (Merged)

In [21]:
# merged = train_df.merge(train_df2, on='tweet')
train_data = train.append(train2, ignore_index=True)
test_data = test.append(test2, ignore_index= True)

In [22]:
sarcasm= train_data.append(test_data, ignore_index= True)
sarcasm.to_csv('Sarcasm.csv')

# Words Tokenization + Split

In [23]:
data= pd.read_csv('/content/drive/My Drive/Bachelor/Cleaned Data/Sarcasm.csv') 

In [24]:
import numpy as np
max_sequence_length = np.max(data['Number of Words in tweet'])
print("max length is:", max_sequence_length)

max length is: 56


In [25]:
text = data['cleaned_tweet']
label = data['sarcasm']

In [26]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(text.astype(str))
sequences = tokenizer.texts_to_sequences(text.astype(str))

word_index = tokenizer.word_index   # a dictionary of each word and its index
print('Found %s unique tokens.' % len(word_index))


from keras_preprocessing.sequence import pad_sequences
data = pad_sequences(sequences, maxlen=max_sequence_length)
print('Shape of data tensor:', data.shape)

Found 55395 unique tokens.
Shape of data tensor: (26095, 56)


In [28]:
label[0:5]

0    False
1    False
2    False
3     True
4    False
Name: sarcasm, dtype: bool

In [29]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(label))  ## one hot of the output
print('Shape of label tensor:', labels.shape)

Shape of label tensor: (26095, 2)


In [30]:
labels[0:5]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [31]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

# Arabert

In [32]:
!pip install bert-for-tf2
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 KB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=4f311b8a062e912485f802316edf5d5ae9aa9295de5df489d715719696fdca7b
  Stored in directory: /root/.cache/pip/wheels/ab/a4/72/df07592cea3ae06b5e846f5e52262f8b16748e829ca354b7df
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=68f77b49f5a764708dcc8d218d5af0162c0b45e7c1283db467e477ede70e8004
  Stored in directory: /root/.cache/pip/wheels/c7/f3/85/b8cf1d8bfe55dc2ece0f1fcd4e91d6f8fc7b59ff3fd75329e1
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=039159670315df674880c2174476dd9021b7813322a881067d97c75ae71f2add
  Sto

In [33]:
import numpy as np
import math
import re
import pandas as pd
import random

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt

In [34]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert
from transformers import AutoTokenizer, AutoModel

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [35]:
arabert_tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv01",do_lower_case=False)
def encode_sentence(sent):
    tokenized = arabert_tokenizer.tokenize(sent)
    return arabert_tokenizer.convert_tokens_to_ids(tokenized)

Downloading:   0%|          | 0.00/379 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.70M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [36]:
VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200